<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b> LSST Crowded Fields: Test <code>crowdSource</code> code on 47 Tuc Data </b><br>
Use LSST ComCam <code>preliminary_visit_images</code> data of 47 Tuc to test <code>crowdSource</code> code. <br> <br>

Contact author: Audrey Budlong <br>
Last verified to run: 16 February 2026 <br>

### Notebook Contents:
1. Install <code>crowdSource</code>
2. Imports
3. Setup
4. Load Data
5. Prepare Input PSF
6. Run <code>crowdSource</code> on Single Visit
7. Results & Comparison
8. Histogram of Results
9. PVI Source Catalog
10. Magnitude Histograms
11. Overlapping Sources
12. Image Cutouts
13. Visualize <code>crowdSource</code> vs. PVI sourceCatalog Results
14. Duplicate Edge Source Removal

### 1.Install <code>crowdSource</code>

In [ ]:
pip install crowdsourcephoto

### 2. Imports

In [ ]:
import crowdsource
import lsst.afw.display as afw_display
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from crowdsource.psf import SimplePSF
from crowdsource.crowdsource_base import fit_im
from lsst.daf.butler import Butler
from lsst.geom import SpherePoint, Angle, Point2D
from lsst.utils.plotting import (
    get_multiband_plot_colors,
    get_multiband_plot_symbols,
    get_multiband_plot_linestyles,
)
from matplotlib.patches import Circle

### 3. Setup

In [ ]:
plt.style.use('seaborn-v0_8-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
plt.rcParams['font.family'] = 'serif'

In [ ]:
filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

In [ ]:
afw_display.setDefaultBackend('firefly')
display = afw_display.Display(frame=1)

### 4. Load Data

In [ ]:
collections = [
                "LSSTComCam/DP1/defaults",
                "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530",
                "skymaps",
            ]

instrument="LSSTComCam"
skymap="lsst_cells_v1"
repo="/repo/main"

butler = Butler(repo, instrument=instrument, collections=collections, skymap=skymap)

In [ ]:
visit = 2024112600111
ccd = 8

In [ ]:
pvi = butler.get("preliminary_visit_image", dataId={"detector": ccd, "visit": visit})
wcs= pvi.wcs

### 5. Prepare Input PSF

In [ ]:
# Prepare your image and weight arrays
image_array = pvi.image.array        # your 2D image
variance_array = pvi.variance.array  # variance array
weight_array = 1.0 / variance_array  # Crowdsource expects "weights" ~ 1/variance

In [ ]:
# Take a look at the `NOT_DEBLENDED` parts of this image
display = afw_display.Display(frame=1)
display.mtv(pvi)

In [ ]:
display2 = afw_display.Display(frame=2)
display2.mtv(pvi.variance)

In [ ]:
# Define center coordinates for PSF
xc = image_array.shape[1] // 2
yc = image_array.shape[0] // 2

# Prepare PSF as a 2D numpy array from the image's PSF
psf_model = pvi.getPsf()
psf_stamp = psf_model.computeImage(Point2D(xc, yc))
psf_array = np.array(psf_stamp.array)

In [ ]:
plt.imshow(psf_array, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.title('PSF from PVI at Center Coordinate')
plt.show()

In [ ]:
# Pad/crop to desired PSF stamp size
size = 41
psf_array = np.pad(psf_array,
                   ((0, max(0, size - psf_array.shape[0])),
                    (0, max(0, size - psf_array.shape[1]))))
psf_array = psf_array[:size, :size]  # make sure it's square and 2D

# Create a SimplePSF object
simple_psf = SimplePSF(psf_array)

In [ ]:
plt.imshow(simple_psf.stamp, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.title('PSF from crowdSource SimplePSF')
plt.show()

In [ ]:
simple_psf.stamp == psf_array

### 6. Run <code>crowdSource</code> on Single Visit

In [ ]:
# Run crowdsource
stars, model_image, sky_image, psf_fitted = fit_im(
    image_array,       # the image to fit
    simple_psf,        # SimplePSF object
    weight=weight_array,
    refit_psf=True,
    verbose=True,
    miniter=5,
    maxiter=10,
    blist=None,
    maxstars=40000,
    derivcentroids=False,
    ntilex=1, ntiley=1,
    fewstars=100,
    # threshold=5,
    threshold=0.1,
    ccd=None, plot=True,
    titer_thresh=2, blendthreshu=2,
    psfvalsharpcutfac=0.7, psfsharpsat=0.7
)

### 7. Results & Comparison

In [ ]:
pvi_image = pvi.image.array
difference_image = pvi_image - model_image

fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)

for ax in axes:
    ax.set_aspect('equal')

# Panel 1: PVI
ax = axes[0]
ax.imshow(pvi_image, origin='lower', cmap='gray',
           vmin=np.percentile(pvi_image, 5),
           vmax=np.percentile(pvi_image, 99))
ax.set_title('PVI')
ax.set_xlabel('x [pixels]')
ax.set_ylabel('y [pixels]')
# ax.legend()

# Panel 2: crowdSource Model Image
ax = axes[1]
ax.imshow(model_image, origin='lower', cmap='gray',
           vmin=np.percentile(model_image, 5),
           vmax=np.percentile(model_image, 99))
ax.set_title('crowdSource Model Image')
ax.set_xlabel('x [pixels]')
# ax.legend()

# Panel 3: crowdSource vs sourceCatalog
ax = axes[2]
ax.imshow(difference_image, origin='lower', cmap='gray',
           vmin=np.percentile(difference_image, 5),
           vmax=np.percentile(difference_image, 99));
ax.set_title('Difference Image: PVI - crowdSource Model Image')
ax.set_xlabel('x [pixels]')
# ax.legend()

plt.show()


In [ ]:
pvi_image = pvi.image.array
plt.imshow(pvi_image, origin='lower', cmap='gray',
           vmin=np.percentile(pvi_image, 5),
           vmax=np.percentile(pvi_image, 99))



In [ ]:
difference_image = pvi_image - model_image
plt.imshow(difference_image, origin='lower', cmap='gray',
           vmin=np.percentile(difference_image, 5),
           vmax=np.percentile(difference_image, 99));


In [ ]:
plt.imshow(model_image, origin='lower', cmap='gray',
           vmin=np.percentile(model_image, 5),
           vmax=np.percentile(model_image, 99))

In [ ]:
plt.imshow(sky_image, origin='lower', cmap='gray',
           vmin=np.percentile(sky_image, 5),
           vmax=np.percentile(sky_image, 99))

### 8. Histogram of Results

In [ ]:
y = stars['x']
x = stars['y']
flux = stars['flux']

In [ ]:
crowdSource_mag = (flux*u.nJy).to(u.ABmag)

In [ ]:
# snr threshold 1
plt.hist(crowdSource_mag.value, label="crowdSource")
plt.ylabel("Counts (#)")
plt.xlabel("AB Magnitude")
plt.title("crowdSource Results: AB Magnitude Histogram")
plt.legend()
plt.show();

In [ ]:
# threshold 0.5
plt.hist(crowdSource_mag.value, label="crowdSource")
plt.ylabel("Counts (#)")
plt.xlabel("AB Magnitude")
plt.title("crowdSource Results: AB Magnitude Histogram")
plt.legend()
plt.show();

### 9. PVI Source Catalog

In [ ]:
# single_visit_star_footprints
src_catalog_original = butler.get("single_visit_star_footprints", dataId={"visit": visit, "detector": ccd})

# src_catalog = butler.get("initial_astrometry_match_detector", dataId={"visit": visit, "detector": ccd}).asAstropy().to_pandas()
src_catalog_original

In [ ]:
lsstSourceCat_flux = src_catalog_original.getPsfInstFlux()
# lsstSourceCat_flux
lsstSource_mag = (lsstSourceCat_flux*u.nJy).to(u.ABmag)
lsstSource_mag

### 10. Magnitude Histograms

In [ ]:
custom_bins = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
counts, bins, patches = plt.hist(lsstSource_mag.value, bins=custom_bins, label=f"LSST: {len(lsstSource_mag)} Total Sources", linewidth=1.5, color='b',alpha=0.8)
for i, patch in enumerate(patches):
    x = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x, y, int(counts[i]), ha='center', va='top', fontsize=7, color='b')


counts2, bins2, patches2 = plt.hist(crowdSource_mag.value, bins=custom_bins, label=f"crowdSource: {len(crowdSource_mag)} Total Sources", linewidth=1.5, color='orange', alpha=0.8)
for i, patch in enumerate(patches2):
    x2 = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y2 = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x2, y2, int(counts2[i]), ha='center', va='bottom', fontsize=7, color='orange')

# plt.ylim(0,550)
plt.ylabel("Counts (#)")
plt.xlabel("AB Magnitude")
plt.title("Comparison AB Magnitude Histogram")
plt.legend()
plt.show();

In [ ]:
custom_bins = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
counts, bins, patches = plt.hist(lsstSource_mag.value, bins=custom_bins, label=f"LSST: {len(lsstSource_mag)} Total Sources", linewidth=1.5, color='b',alpha=0.8)
for i, patch in enumerate(patches):
    x = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x, y, int(counts[i]), ha='center', va='top', fontsize=7, color='b')

plt.ylim(0,550)
plt.ylabel("Counts (#)")
plt.xlabel("AB Magnitude")
plt.title("LSST AB Magnitude Histogram")
plt.legend()
plt.show();

In [ ]:
custom_bins = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
counts2, bins2, patches2 = plt.hist(crowdSource_mag.value, bins=custom_bins, label=f"crowdSource: {len(crowdSource_mag)} Total Sources", linewidth=1.5, color='orange', alpha=0.8)
for i, patch in enumerate(patches2):
    x2 = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y2 = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x2, y2, int(counts2[i]), ha='center', va='bottom', fontsize=7, color='orange')

plt.ylim(0,2700)
plt.ylabel("Counts (#)")
plt.xlabel("AB Magnitude")
plt.title("crowdSource AB Magnitude Histogram")
plt.legend()
plt.show();

In [ ]:
# custom_bins = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
cbin = np.linspace(21.5, 23, 10)
counts2, bins2, patches2 = plt.hist(crowdSource_mag.value, bins=cbin, label=f"crowdSource: {len(crowdSource_mag)} Total Sources", linewidth=1.5, color='orange', alpha=0.8)
for i, patch in enumerate(patches2):
    x2 = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y2 = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x2, y2, int(counts2[i]), ha='center', va='bottom', fontsize=7, color='orange')

plt.ylim(0,1500)
plt.ylabel("Counts (#)")
plt.xlabel("AB Magnitude")
plt.title("crowdSource AB Magnitude Histogram")
plt.legend()
plt.show();

In [ ]:
# Calculate the signal to noise ratio

snr = src_catalog_original.getPsfInstFlux()/src_catalog_original.getPsfInstFluxErr()

src_catalog_original = src_catalog_original.asAstropy().to_pandas()

src_catalog_original["snr"] = snr
src_catalog_original

In [ ]:
for column in src_catalog_original.columns:
    if "flag" in column:
        print(column)

In [ ]:
src_catalog = src_catalog_original[(src_catalog_original["sky_source"]==False) & (src_catalog_original["snr"]>=5)]
src_catalog

In [ ]:
for column in src_catalog.columns:
    # print(column)
    if "flag" in column:
        print(column)

In [ ]:
# Convert catalog RA/Dec to SpherePoint objects
sky_points = [SpherePoint(Angle(ra), Angle(dec)) 
              for ra, dec in zip(src_catalog['coord_ra'], src_catalog['coord_dec'])]

# Convert to pixel coordinates
pixel_points = wcs.skyToPixel(sky_points)  # returns list of Point2D

# Extract x and y
x_cat = np.array([p.getX() for p in pixel_points])
y_cat = np.array([p.getY() for p in pixel_points])

In [ ]:
crowd_source_sky_coords = [wcs.pixelToSky(x_pixel, y_pixel) for (x_pixel, y_pixel) in zip()]

In [ ]:
sourceCat_data = {'x': x_cat, 'y': y_cat}
sourceCat_pixel = pd.DataFrame(sourceCat_data)

In [ ]:
sourceCat_pixel

In [ ]:
y = stars['x']
x = stars['y']
flux = stars['flux']

In [ ]:
crowdCat_data = {'x': x, 'y': y}
crowdCat_pixel = pd.DataFrame(crowdCat_data)

In [ ]:
crowdCat_pixel

In [ ]:
crowd_source_sky_coords = [wcs.pixelToSky(x_pixel, y_pixel) for (x_pixel, y_pixel) in zip(crowdCat_pixel['x'], crowdCat_pixel['y'])]
crowd_source_sky_coords

In [ ]:
crowd_source_ra = [sp.getRa().asRadians() for sp in crowd_source_sky_coords]
crowd_source_dec = [sp.getDec().asRadians() for sp in crowd_source_sky_coords]

### 11. Overlapping Sources

In [ ]:
crowd_source_catalog_data = {"coord_ra": crowd_source_ra, "coord_dec": crowd_source_dec}
crowd_source_catalog = pd.DataFrame(crowd_source_catalog_data)
crowd_source_catalog

In [ ]:
crowd_source_coords = SkyCoord(
    ra=crowd_source_catalog['coord_ra'],
    dec=crowd_source_catalog['coord_dec'],
    unit=u.rad,
    frame='icrs'
)

source_cat_coords = SkyCoord(
    ra=src_catalog['coord_ra'],
    dec=src_catalog['coord_dec'],
    unit=u.rad,
    frame='icrs'
)


In [ ]:
idx, d2d, d3d = source_cat_coords.match_to_catalog_sky(crowd_source_coords)

In [ ]:
d2d_arcsec = [d.to(u.arcsec) for d in d2d]
custom_bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
counts, bins, patches = plt.hist(d2d_arcsec, bins=custom_bins, color='orange', label='On-Sky Separations')
for i, patch in enumerate(patches):
    x = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x, y, int(counts[i]), ha='center', va='top', fontsize=7, color='orange')
plt.ylabel("Counts")
plt.xlabel("Separation (arcsec)")
plt.title("Separation Between LSST Source Catalog \n & crowdSource RA/DEC Values")
# plt.xlim(-0.0001,0.0005)
plt.ylim(0,1300)
plt.legend();

In [ ]:
d2d_arcsec = [d.to(u.arcsec) for d in d2d]
custom_bins=np.linspace(0,1,10)
counts, bins, patches = plt.hist(d2d_arcsec, bins=custom_bins, color='orange', label='On-Sky Separations')
for i, patch in enumerate(patches):
    x = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x, y, int(counts[i]), ha='center', va='top', fontsize=7, color='orange')
plt.ylabel("Counts")
plt.xlabel("Separation (arcsec)")
plt.title("Separation Between LSST Source Catalog \n & crowdSource RA/DEC Values")
# plt.xlim(-0.0001,0.0005)
plt.ylim(0,1000)
plt.legend();

In [ ]:
d2d_arcsec = [d.to(u.arcsec) for d in d2d]
custom_bins=np.linspace(0,0.1,10)
counts, bins, patches = plt.hist(d2d_arcsec, bins=custom_bins, color='orange', label='On-Sky Separations')
for i, patch in enumerate(patches):
    x = patch.get_x() + patch.get_width() / 2  # Center of the bar
    y = patch.get_height() + 0.05 * np.max(counts) # Slightly above the bar top
    plt.text(x, y, int(counts[i]), ha='center', va='top', fontsize=7, color='orange')
plt.ylabel("Counts")
plt.xlabel("Separation (arcsec)")
plt.title("Separation Between LSST Source Catalog \n & crowdSource RA/DEC Values")
# plt.xlim(-0.0001,0.0005)
plt.ylim(0,400)
plt.legend();

In [ ]:
matched_cat = crowd_source_catalog.iloc[idx]
len(matched_cat)

In [ ]:
all_indices = set(range(len(crowd_source_catalog)))
matched = set(idx)
unmatched = all_indices - matched
unmatched_indices = sorted(unmatched)


In [ ]:
unmatched_cat = crowd_source_catalog.iloc[unmatched_indices]
len(unmatched_cat)

In [ ]:
plt.scatter(matched_cat["coord_ra"], matched_cat["coord_dec"], edgecolor='orange', facecolor='orange')
plt.scatter(src_catalog["coord_ra"], src_catalog["coord_dec"], edgecolor='b', facecolor='none')

In [ ]:
def skyToPixelConversion(coordRa, coordDec):
    sky_points = [SpherePoint(Angle(ra), Angle(dec)) 
                  for ra, dec in zip(coordRa, coordDec)]
    pixel_points = wcs.skyToPixel(sky_points)  # returns list of Point2D
    x_vals = np.array([p.getX() for p in pixel_points])
    y_vals = np.array([p.getY() for p in pixel_points])
    return x_vals, y_vals

In [ ]:
crowd_source_matched_x, crowd_source_matched_y =  skyToPixelConversion(matched_cat["coord_ra"], matched_cat["coord_dec"])
source_cat_x, source_cat_y = skyToPixelConversion(src_catalog["coord_ra"], src_catalog["coord_dec"])
crowd_source_unmatched_x, crowd_source_unmatched_y = skyToPixelConversion(unmatched_cat["coord_ra"], unmatched_cat["coord_dec"])

In [ ]:
cmap = 'rainbow'

pvi_image = pvi.image.array

ny, nx = pvi_image.shape

fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)

for ax in axes:
    ax.set_aspect('equal')

# Panel 1: PVI + crowdSource
ax = axes[0]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc1 = ax.scatter(crowd_source_matched_x, crowd_source_matched_y, facecolors='none', edgecolors='orange',
                label='crowdSource')
ax.set_title('PVI + crowdSource')
ax.set_xlabel('x [pixels]')
ax.set_ylabel('y [pixels]')
ax.legend()

# Panel 2: PVI + sourceCatalog
ax = axes[1]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc2 = ax.scatter(source_cat_x, source_cat_y, facecolors='none', edgecolors='blue',
                 label='LSST sourceCatalog')
ax.set_title('PVI + LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

# Panel 3: crowdSource vs sourceCatalog
ax = axes[2]
ax.scatter(crowd_source_matched_x, crowd_source_matched_y, facecolors='orange', edgecolors='orange',
           label='crowdSource', alpha=0.7)
ax.scatter(source_cat_x, source_cat_y, facecolors='none', edgecolors='blue',
           label='LSST sourceCatalog', alpha=0.7)
ax.set_title('crowdSource vs LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

plt.show()


In [ ]:
cmap = 'rainbow'

pvi_image = pvi.image.array

ny, nx = pvi_image.shape

fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)

for ax in axes:
    ax.set_aspect('equal')

# Panel 1: PVI + crowdSource
ax = axes[0]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc1 = ax.scatter(crowd_source_matched_x, crowd_source_matched_y, facecolors='none', edgecolors='orange',
                label='matched crowdSource')
ax.set_title('PVI + crowdSource')
ax.set_xlabel('x [pixels]')
ax.set_ylabel('y [pixels]')
ax.legend()

# Panel 2: PVI + sourceCatalog
ax = axes[1]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc2 = ax.scatter(crowd_source_unmatched_x, crowd_source_unmatched_y, facecolors='none', edgecolors='red',
                 label='unmatched crowdSource')
ax.set_title('PVI + crowdSource')
ax.set_xlabel('x [pixels]')
ax.legend()

# Panel 3: crowdSource vs sourceCatalog
ax = axes[2]
ax.scatter(crowd_source_matched_x, crowd_source_matched_y, facecolors='orange', edgecolors='orange',
                label='matched crowdSource')
ax.scatter(crowd_source_unmatched_x, crowd_source_unmatched_y, marker='x', color='red', # facecolors='none', edgecolors='red',
                 label='unmatched crowdSource')
ax.set_title('crowdSource vs LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

plt.show()


### 12. Image Cutouts

In [ ]:
def cutout_from_pixels(pvi, x0, y0, postage_size=100, radius=10):
    """
    pvi  : LSST exposure with .image.array
    x0,y0: center in pixel coordinates (floats or ints)
    """
    img = pvi.image.array
    ymax, xmax = img.shape

    x0 = int(x0)
    y0 = int(y0)

    # handle edges
    size = min(postage_size,
               x0, y0,
               xmax - x0,
               ymax - y0)

    cutout = img[y0-size:y0+size, x0-size:x0+size]

    fig, ax = plt.subplots(figsize=(4,4))
    ax.imshow(cutout, origin="lower", vmin=-50, vmax=100)

    # mark the center with a circle
    ax.add_patch(Circle((size, size), radius,
                        edgecolor="yellow", facecolor="none", linewidth=1.5))

    plt.savefig(f"cutout_v1_{x0}_{y0}.png")
    plt.show()


In [ ]:
for x0, y0 in zip(crowd_source_matched_x[:20], crowd_source_matched_y[:20]):
    cutout_from_pixels(pvi, x0, y0)


In [ ]:
for x0, y0 in zip(crowd_source_unmatched_x[:20], crowd_source_unmatched_y[:20]):
    cutout_from_pixels(pvi, x0, y0)


In [ ]:
def cutout_with_overlay(pvi, x0, y0,
                        overlay_x=None, overlay_y=None,
                        postage_size=100, radius=10):
    """
    pvi        : Exposure with .image.array
    x0, y0     : center pixel coords (float or int)
    overlay_x,
    overlay_y  : arrays of secondary catalog pixel coords (optional)
    """

    img = pvi.image.array
    ymax, xmax = img.shape

    x0 = int(x0)
    y0 = int(y0)

    # handle edges
    size = min(postage_size,
               x0, y0,
               xmax - x0,
               ymax - y0)

    # extract postage cutout
    cutout = img[y0-size:y0+size, x0-size:x0+size]

    fig, ax = plt.subplots(figsize=(4,4))
    ax.imshow(cutout, origin="lower", vmin=-50, vmax=100)

    # draw center marker
    ax.add_patch(Circle((size, size), radius,
                        edgecolor="yellow", facecolor="none", linewidth=1.5))

    # overlay the secondary catalog if provided
    if overlay_x is not None and overlay_y is not None:
        # translate full-frame coords → cutout coords
        xcut = overlay_x - (x0 - size)
        ycut = overlay_y - (y0 - size)

        # keep only points within the cutout
        inside = (
            (0 < xcut) & (xcut < 2*size) &
            (0 < ycut) & (ycut < 2*size)
        )

        for xc, yc in zip(xcut[inside], ycut[inside]):
            ax.add_patch(Circle((xc, yc), radius,
                                edgecolor="cyan", facecolor="none", linewidth=1))
    plt.savefig(f"cutout_{x0}_{y0}.png")
    plt.show()


In [ ]:
lim_min = 0
lim_max= 20
for x0, y0, x1, y1 in zip(crowd_source_matched_x[lim_min:lim_max], crowd_source_matched_y[lim_min:lim_max],
                         source_cat_x[lim_min:lim_max], source_cat_y[lim_min:lim_max]):
    cutout_with_overlay(
        pvi,
        x0, y0,
        overlay_x=x1,
        overlay_y=y1,
        postage_size=100,
        radius=10
    )


### 13. Visualize <code>crowdSource</code> vs. PVI sourceCatalog Results

In [ ]:
cmap = 'rainbow'

pvi_image = pvi.image.array

ny, nx = pvi_image.shape

fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)

for ax in axes:
    ax.set_aspect('equal')

# Panel 1: PVI + crowdSource
ax = axes[0]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc1 = ax.scatter(x, y, facecolors='none', edgecolors='orange',
                label='crowdSource')
ax.set_title('PVI + crowdSource')
ax.set_xlabel('x [pixels]')
ax.set_ylabel('y [pixels]')
ax.legend()

# Panel 2: PVI + sourceCatalog
ax = axes[1]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc2 = ax.scatter(x_cat, y_cat, facecolors='none', edgecolors='blue',
                 label='LSST sourceCatalog')
ax.set_title('PVI + LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

# Panel 3: crowdSource vs sourceCatalog
ax = axes[2]
ax.scatter(x, y, facecolors='orange', edgecolors='orange',
           label='crowdSource', alpha=0.7)
ax.scatter(x_cat, y_cat, facecolors='none', edgecolors='blue',
           label='LSST sourceCatalog', alpha=0.7)
ax.set_title('crowdSource vs LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()
plt.show()


### 14. Duplicate Edge Source Removal

In [ ]:
test_reduced_crowd_cat = crowdCat_pixel.drop_duplicates()

In [ ]:
len(crowdCat_pixel), len(test_reduced_crowd_cat)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
sorted_df = crowdCat_pixel.sort_values(by=['x', 'y'])
sorted_df

In [ ]:
test1 = sorted_df[(sorted_df["x"]>=0) & (sorted_df["x"]<=4071.)]
test1

In [ ]:
cmap = 'rainbow'

pvi_image = pvi.image.array

ny, nx = pvi_image.shape

fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)

for ax in axes:
    ax.set_aspect('equal')

x = test1["x"]
y = test1["y"]

# Panel 1: PVI + crowdSource
ax = axes[0]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc1 = ax.scatter(x, y, facecolors='none', edgecolors='orange',
                label='crowdSource')
ax.set_title('PVI + crowdSource')
ax.set_xlabel('x [pixels]')
ax.set_ylabel('y [pixels]')
ax.legend()

# Panel 2: PVI + sourceCatalog
ax = axes[1]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc2 = ax.scatter(x_cat, y_cat, facecolors='none', edgecolors='blue',
                 label='LSST sourceCatalog')
ax.set_title('PVI + LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

# Panel 3: crowdSource vs sourceCatalog
ax = axes[2]
ax.scatter(x, y, facecolors='orange', edgecolors='orange',
           label='crowdSource', alpha=0.7)
ax.scatter(x_cat, y_cat, facecolors='none', edgecolors='blue',
           label='LSST sourceCatalog', alpha=0.7)
ax.set_title('crowdSource vs LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()
plt.show()


In [ ]:
cmap = 'rainbow'

pvi_image = pvi.image.array

ny, nx = pvi_image.shape

fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)

for ax in axes:
    ax.set_aspect('equal')

x = test1["x"]
y = test1["y"]

# Panel 1: PVI + crowdSource
ax = axes[0]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc1 = ax.scatter(x, y, facecolors='none', edgecolors='orange',
                label='crowdSource')
ax.set_title('PVI + crowdSource')
ax.set_xlabel('x [pixels]')
ax.set_ylabel('y [pixels]')
ax.legend()

# Panel 2: PVI + sourceCatalog
ax = axes[1]
ax.imshow(pvi_image, origin='lower', cmap='gray',
          vmin=np.percentile(pvi_image, 5),
          vmax=np.percentile(pvi_image, 99),
          extent=[0, nx, 0, ny])
sc2 = ax.scatter(x_cat, y_cat, facecolors='none', edgecolors='blue',
                 label='LSST sourceCatalog')
ax.set_title('PVI + LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

# Panel 3: crowdSource vs sourceCatalog
ax = axes[2]
ax.scatter(x, y, facecolors='orange', edgecolors='orange',
           label='crowdSource', alpha=0.7)
ax.scatter(x_cat, y_cat, facecolors='none', edgecolors='blue',
           label='LSST sourceCatalog', alpha=0.7)
ax.set_title('crowdSource vs LSST sourceCatalog')
ax.set_xlabel('x [pixels]')
ax.legend()

plt.show()
